## Travelling Salesman problem

In [11]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

In [12]:
!pip install ortools

In [43]:

from IPython.display import display, HTML
from dimod import BinaryQuadraticModel
import random
import numpy as np
import os
import itertools
import time
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

random.seed(42)
ccities = [
    "Munich", "Hamburg", "Stuttgart", "Berlin", "Frankfurt", 
    "Paris", "Madrid", "Barcelona", "Tenerife", "Malaga", 
    "Valencia", "Sevilla", "Lisboa", "Roma"
]


n = len(cities)
A = 10
B = 10
C = 1
distances = {}
for cindex1 in range(len(cities)-1):
    c1 = cities[cindex1]
    for cindex2 in range(cindex1+1,len(cities)):
        c2 = cities[cindex2]
        distances[c1,c2] = random.randint(1,6)
        distances[c2,c1] = distances[c1,c2]

#distances2 = {}
#distances2 = np.zeros((n, n), dtype=int)
#for i in range(n):
 #   for j in range(i + 1, n):
#        distances2[i, j] = distances2[j, i] = random.randint(1, 6)

#print("Distance Matrix:")
#print(distances2)

print(distances)

distance_matrix = np.zeros((n, n), dtype=int)
for (c1, c2), dist in distances.items():
    i, j = cities.index(c1), cities.index(c2)
    distance_matrix[i, j] = distance_matrix[j, i] = dist

print("Distance Matrix:")
print(distance_matrix)


{('Munich', 'Hamburg'): 6, ('Hamburg', 'Munich'): 6, ('Munich', 'Stuttgart'): 1, ('Stuttgart', 'Munich'): 1, ('Munich', 'Berlin'): 1, ('Berlin', 'Munich'): 1, ('Munich', 'Frankfurt'): 6, ('Frankfurt', 'Munich'): 6, ('Munich', 'Paris'): 3, ('Paris', 'Munich'): 3, ('Munich', 'Madrid'): 2, ('Madrid', 'Munich'): 2, ('Munich', 'Barcelona'): 2, ('Barcelona', 'Munich'): 2, ('Munich', 'Tenerife'): 2, ('Tenerife', 'Munich'): 2, ('Munich', 'Malaga'): 6, ('Malaga', 'Munich'): 6, ('Munich', 'Valencia'): 1, ('Valencia', 'Munich'): 1, ('Munich', 'Sevilla'): 6, ('Sevilla', 'Munich'): 6, ('Munich', 'Lisboa'): 6, ('Lisboa', 'Munich'): 6, ('Munich', 'Roma'): 5, ('Roma', 'Munich'): 5, ('Munich', 'London'): 1, ('London', 'Munich'): 1, ('Munich', 'Brussels'): 5, ('Brussels', 'Munich'): 5, ('Munich', 'Amsterdam'): 4, ('Amsterdam', 'Munich'): 4, ('Munich', 'Zurich'): 1, ('Zurich', 'Munich'): 1, ('Munich', 'Vienna'): 1, ('Vienna', 'Munich'): 1, ('Munich', 'Prague'): 1, ('Prague', 'Munich'): 1, ('Munich', 'War

### Creates el data model

In [44]:

# Utilizando RoutingModel de OR-Tools
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


### Find the minima using RoutingModel

In [45]:
import time
import psutil
N = len(cities)
T = N
def get_memory_usage():
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024  # Convertir a MB
def print_solution(manager, routing, solution):
    print('Objective: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    route = []
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(cities[manager.IndexToNode(index)])
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        route.append(manager.IndexToNode(previous_index))
    route.append(manager.IndexToNode(index))
    plan_output += ' {}\n'.format(cities[manager.IndexToNode(index)])
    plan_output += 'Route distance: {}\n'.format(route_distance)
    print(plan_output)
    return route, route_distance

def main():
    data = create_data_model()

    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    initial_memory = get_memory_usage()

    start_time = time.time()

    solution = routing.SolveWithParameters(search_parameters)
    end_time = time.time()
    final_memory = get_memory_usage()

    if solution:
        route, route_distance = print_solution(manager, routing, solution)
        print("La ruta óptima es:", [cities[i] for i in route])
        print("Distancia total recorrida:", route_distance)
    else:
        print("No se encontró solución óptima.")
        
    print("Tiempo de ejecución:", end_time - start_time, "segundos")
    print("Uso de memoria inicial:", initial_memory, "MB")
    print("Uso de memoria final:", final_memory, "MB")
    print("Incremento de uso de memoria:", final_memory - initial_memory, "MB")

if __name__ == '__main__':
    main()

Objective: 94
Route:
 Munich -> The Hague -> Saint Petersburg -> Krakow -> Zagreb -> Poznan -> Lodz -> Belgrade -> Podgorica -> Ljubljana -> Minsk -> Chisinau -> Bucharest -> Odessa -> Alicante -> Marseille -> Naples -> Lyon -> Warsaw -> Rotterdam -> Innsbruck -> Paris -> Frankfurt -> Helsinki -> Reykjavik -> Sevilla -> Zurich -> Copenhagen -> Toulouse -> Stockholm -> Nice -> Santander -> Dublin -> Budapest -> Bordeaux -> Bruges -> Vienna -> Prague -> Venice -> Zaragoza -> Amsterdam -> Berlin -> Lisboa -> Oslo -> Madrid -> London -> Roma -> Tenerife -> Barcelona -> Sarajevo -> Gdansk -> Bilbao -> Wroclaw -> Granada -> Skopje -> Geneva -> Turin -> Lucerne -> Ghent -> Basel -> Antwerp -> Vilnius -> Sofia -> Florence -> Milan -> Salzburg -> Hamburg -> Brussels -> Malaga -> Stuttgart -> Athens -> Glasgow -> Tirana -> Tallinn -> Moscow -> Vaduz -> Bratislava -> Valletta -> Sibiu -> Luxembourg -> Kyiv -> Nicosia -> Edinburgh -> Split -> Monaco -> San Marino -> Cardiff -> Riga -> Andorra la V

### Draw Solution

In [12]:
import matplotlib.pyplot as plt
import numpy as np

# Función para visualizar la ruta y las distancias entre ciudades
def plot_route(route, cities, distance_matrix):
    n = len(cities)
    plt.figure(figsize=(10, 10))  # Aumentar el tamaño del gráfico
    ax = plt.gca()

    # Coordenadas de las ciudades (para visualización)
    coords = np.random.rand(n, 2) * 100  # Coordenadas aleatorias en un plano 100x100
    for i, (x, y) in enumerate(coords):
        ax.plot(x, y, 'o', markersize=10, label=cities[i])
        ax.text(x, y, f' {cities[i]}', fontsize=12)

    # Dibujar la ruta y añadir las distancias
    total_distance = 0
    for i in range(n):
        start_city = route[i]
        end_city = route[(i + 1) % n]
        start_coord = coords[start_city]
        end_coord = coords[end_city]
        ax.plot([start_coord[0], end_coord[0]], [start_coord[1], end_coord[1]], 'k-')
        
        # Calcular el punto medio para poner la etiqueta de la distancia
        mid_point = (start_coord + end_coord) / 2
        distance = distance_matrix[start_city, end_city]
        total_distance += distance
        
        # Ajustar posición de la etiqueta para que no se solape
        label_offset = np.array([0, 0])  # Ajuste vertical
        if start_coord[0] > end_coord[0]:
            label_offset[0] = -15  # Ajuste horizontal si necesario
        ax.text(mid_point[0] + label_offset[0], mid_point[1] + label_offset[1], 
                f'{distance}', color='red', fontsize=10, ha='center')

    # Dibujar la línea de regreso a la ciudad inicial
    start_city = route[0]
    start_coord = coords[start_city]
    end_coord = coords[route[-1]]
    ax.plot([start_coord[0], end_coord[0]], [start_coord[1], end_coord[1]], 'k-')

    plt.title(f"Ruta Óptima del Viajante - Distancia Total: {total_distance}")
    plt.legend()
    plt.grid(True)
    plt.show()

# Llamar a la función para dibujar la ruta con distancias
plot_route(sorted_route, cities, distance_matrix)


NameError: name 'sorted_route' is not defined